In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import

In [2]:
import random
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import os
import re
import glob
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

In [3]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [30]:
CFG = {
    'IMG_SIZE':224,
    'EPOCHS':30,
    'LEARNING_RATE':0.5, #3e-4,
    'BATCH_SIZE':128,
    'SEED':41
}

## Fixed RandomSeed

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-processing

In [6]:
all_img_list = glob.glob('drive/MyDrive/Doedal/train/*/*') 

In [ ]:
all_img_list

['drive/MyDrive/Doedal/train/걸레받이수정/111.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/11.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/175.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/178.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/20.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/87.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/28.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/72.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/94.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/26.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/164.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/71.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/239.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/209.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/159.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/57.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/235.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/98.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/174.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/118.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/86.png',
 'drive/MyDrive/Doedal/train/걸레받이수정/198.png',

In [7]:
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[4])

In [8]:
df['label'].value_counts()

훼손                 1405
오염                  595
걸레받이수정        307
꼬임                  210
터짐                  162
곰팡이               145
오타공                142
몰딩수정            130
면불량               99
석고수정              57
들뜸                  54
피스                    51
창틀,문틀수정      27
울음                  22
이음부불량           17
녹오염                14
가구수정               12
틈새과다                5
반점                   3
Name: label, dtype: int64

In [9]:
# 라벨 인코더 생성
encoder = LabelEncoder()

# X_train데이터를 이용 피팅하고 라벨숫자로 변환
encoder.fit(df['label'])
new_label = encoder.transform(df['label'])

In [10]:
new_label

array([ 1,  1,  1, ..., 16, 16, 16])

In [11]:
df['label'] = new_label
df

,img_path,label
0,drive/MyDrive/Doedal/train/걸레받이수정/111...,1
1,drive/MyDrive/Doedal/train/걸레받이수정/11.png,1
2,drive/MyDrive/Doedal/train/걸레받이수정/175...,1
3,drive/MyDrive/Doedal/train/걸레받이수정/178...,1
4,drive/MyDrive/Doedal/train/걸레받이수정/20.png,1
...,...,...
3452,drive/MyDrive/Doedal/train/틈새과다/1.png,16
3453,drive/MyDrive/Doedal/train/틈새과다/4.png,16
3454,drive/MyDrive/Doedal/train/틈새과다/3.png,16
3455,drive/MyDrive/Doedal/train/틈새과다/2.png,16


In [12]:
# train 7 : val 3 
train, val, _, _ = train_test_split(df, df['label'], test_size=0.3, stratify=df['label'], random_state=CFG['SEED'])

## Label-Encoding

In [13]:
le = preprocessing.LabelEncoder()
train['label'] = le.fit_transform(train['label'])
val['label'] = le.transform(val['label'])

In [14]:
train['label']

3064    18
1831    18
2063    18
2341    18
2047    18
        ..
2687    18
1495    10
980      6
2509    18
833      7
Name: label, Length: 2419, dtype: int64

In [15]:
val['label']

121      1
2893    18
0        1
1593    10
3095    18
        ..
1184    15
1393    10
2060    18
545      3
1947    18
Name: label, Length: 1038, dtype: int64

## CustomDataset

In [16]:
class CustomDataset(Dataset):
    def __init__(self, img_path_list, label_list, transforms=None):
        self.img_path_list = img_path_list
        self.label_list = label_list
        self.transforms = transforms
        
    def __getitem__(self, index):
        img_path = self.img_path_list[index]
        
        image = cv2.imread(img_path)
        
        if self.transforms is not None:
            image = self.transforms(image=image)['image']
        
        if self.label_list is not None:
            label = self.label_list[index]
            return image, label
        else:
            return image
        
    def __len__(self):
        return len(self.img_path_list)

In [27]:
train_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            A.HorizontalFlip(p=0.5),
                            #A.VerticalFlip(p=0.5),
                            #A.RandomBrightnessContrast(p=0.2),
                            ToTensorV2()
                            ])

test_transform = A.Compose([
                            A.Resize(CFG['IMG_SIZE'],CFG['IMG_SIZE']),
                            A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, always_apply=False, p=1.0),
                            #A.HorizontalFlip(p=0.3),
                            #A.RandomBrightnessContrast(p=0.2),
                            #A.VerticalFlip(p=0.5),
                            ToTensorV2()
                            ])

In [18]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

val_dataset = CustomDataset(val['img_path'].values, val['label'].values, test_transform)
val_loader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

## Model Define

In [31]:
class BaseModel(nn.Module):
    def __init__(self, num_classes=len(le.classes_)):
        super(BaseModel, self).__init__()
        self.backbone = models.efficientnet_b0(pretrained=True)
        self.classifier = nn.Linear(1000, num_classes)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class FocalLoss(nn.Module):
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha]*19)
        self.alpha[18] = 1-alpha
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, input, target):
        if input.dim()>2:
            input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
            input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
            input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
        target = target.view(-1,1)

        logpt = F.log_softmax(input)
        logpt = logpt.gather(1,target)
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=input.data.type():
                self.alpha = self.alpha.type_as(input.data)
            at = self.alpha.gather(0,target.data.view(-1))
            logpt = logpt * at

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

## Train

In [33]:
def train(model, optimizer, train_loader, val_loader, scheduler, device):
    model.to(device)
    #criterion = nn.CrossEntropyLoss().to(device)
    criterion = FocalLoss(gamma=2, alpha=0.25)
    
    best_score = 0
    best_model = None
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        for imgs, labels in tqdm(iter(train_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            output = model(imgs)
            loss = criterion(output, labels)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
                    
        _val_loss, _val_score = validation(model, criterion, val_loader, device)
        _train_loss = np.mean(train_loss)
        print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val Weighted F1 Score : [{_val_score:.5f}]')
       
        if scheduler is not None:
            scheduler.step(_val_score)
            
        if best_score < _val_score:
            best_score = _val_score
            best_model = model
    
    return best_model

In [34]:
def validation(model, criterion, val_loader, device):
    model.eval()
    val_loss = []
    preds, true_labels = [], []

    with torch.no_grad():
        for imgs, labels in tqdm(iter(val_loader)):
            imgs = imgs.float().to(device)
            labels = labels.to(device)
            
            pred = model(imgs)
            
            loss = criterion(pred, labels)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
            true_labels += labels.detach().cpu().numpy().tolist()
            
            val_loss.append(loss.item())
        
        _val_loss = np.mean(val_loss)
        _val_score = f1_score(true_labels, preds, average='weighted')
    
    return _val_loss, _val_score

In [ ]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2, threshold_mode='abs', min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [1], Train Loss : [7402.78915] Val Loss : [1422.94489] Val Weighted F1 Score : [0.23495]


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

...

Epoch [11], Train Loss : [18.04766] Val Loss : [12.13048] Val Weighted F1 Score : [0.03475]
Epoch 00011: reducing learning rate of group 0 to 6.2500e-02.


  0%|          | 0/76 [00:00<?, ?it/s]

  0%|          | 0/33 [00:00<?, ?it/s]

Epoch [12], Train Loss : [15.49423] Val Loss : [11.91047] Val Weighted F1 Score : [0.23502]


  0%|          | 0/76 [00:00<?, ?it/s]

## Inference

In [ ]:
test = pd.read_csv('drive/MyDrive/Doedal/test.csv')
test['img_path'] = test['img_path'].apply(lambda x: x.replace('./', 'drive/MyDrive/Doedal/', 1))

In [ ]:
test

,id,img_path
0,TEST_000,drive/MyDrive/Doedal/test/000.png
1,TEST_001,drive/MyDrive/Doedal/test/001.png
2,TEST_002,drive/MyDrive/Doedal/test/002.png
3,TEST_003,drive/MyDrive/Doedal/test/003.png
4,TEST_004,drive/MyDrive/Doedal/test/004.png
...,...,...
787,TEST_787,drive/MyDrive/Doedal/test/787.png
788,TEST_788,drive/MyDrive/Doedal/test/788.png
789,TEST_789,drive/MyDrive/Doedal/test/789.png
790,TEST_790,drive/MyDrive/Doedal/test/790.png


In [ ]:
test_dataset = CustomDataset(test['img_path'].values, None, test_transform)
test_loader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    model.eval()
    preds = []
    with torch.no_grad():
        for imgs in tqdm(iter(test_loader)):
            imgs = imgs.float().to(device)
            
            pred = model(imgs)
            
            preds += pred.argmax(1).detach().cpu().numpy().tolist()
    
    preds = le.inverse_transform(preds)
    return preds

In [ ]:
preds = inference(infer_model, test_loader, device)

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
preds

array([18, 10, 18, 18, 10, 18, 10, 18, 18, 11,  7, 11, 10, 10, 18, 15,  9,
        1, 10, 18, 14, 18, 18, 15, 18, 18, 10,  2, 18, 18, 10, 18, 18, 10,
       10, 18, 18, 18, 11, 18, 10,  5, 18, 15, 18,  6, 10, 18, 10, 10, 18,
       18, 10, 18, 18, 18, 18, 10, 18, 10, 11,  7, 18, 10, 18, 18, 10, 10,
       18, 15, 10, 10, 10,  4, 18, 18, 18, 10, 18, 15, 10, 18, 18, 11, 15,
       18, 18, 15, 18, 18, 10, 18, 18, 10, 18, 18, 18, 11, 11, 18, 17, 18,
       18, 15, 18, 18,  0, 18, 18, 18, 15, 18, 18, 18, 10, 18, 18, 10, 15,
       18, 18, 10, 18,  2, 18, 17, 18, 18, 18, 10, 18, 18,  9, 10, 11, 10,
       15, 18, 18, 10, 11, 18, 10, 10, 11, 18, 18, 18, 18, 18, 18, 18, 18,
       18, 18, 13, 10, 10, 18, 18, 10, 10, 18, 18, 18, 10, 18, 18, 18, 18,
       18, 18, 18, 10, 10,  3, 10,  9, 18, 14, 18,  2, 11,  3, 18, 18,  3,
       18,  3,  3, 18, 18, 18, 18, 18,  2, 18, 13, 18,  7, 18,  3, 10,  5,
        3,  3, 18, 18,  3, 18,  3, 18, 18, 18, 10,  3, 18, 18,  6, 18, 10,
       18, 15,  2, 18, 10

In [ ]:
result = encoder.inverse_transform(preds)

In [ ]:
result

array(['훼손', '오염', '훼손', '훼손', '오염', '훼손', '오염',
       '훼손', '훼손', '오타공', '몰딩수정', '오타공', '오염',
       '오염', '훼손', '터짐', '석고수정', '걸레받이수정',
       '오염', '훼손', '창틀,문틀수정', '훼손', '훼손', '터짐',
       '훼손', '훼손', '오염', '곰팡이', '훼손', '훼손', '오염',
       '훼손', '훼손', '오염', '오염', '훼손', '훼손', '훼손',
       '오타공', '훼손', '오염', '들뜸', '훼손', '터짐', '훼손',
       '면불량', '오염', '훼손', '오염', '오염', '훼손', '훼손',
       '오염', '훼손', '훼손', '훼손', '훼손', '오염', '훼손',
       '오염', '오타공', '몰딩수정', '훼손', '오염', '훼손',
       '훼손', '오염', '오염', '훼손', '터짐', '오염', '오염',
       '오염', '녹오염', '훼손', '훼손', '훼손', '오염', '훼손',
       '터짐', '오염', '훼손', '훼손', '오타공', '터짐', '훼손',
       '훼손', '터짐', '훼손', '훼손', '오염', '훼손', '훼손',
       '오염', '훼손', '훼손', '훼손', '오타공', '오타공', '훼손',
       '피스', '훼손', '훼손', '터짐', '훼손', '훼손', '가구수정',
       '훼손', '훼손', '훼손', '터짐', '훼손', '훼손', '훼손',
       '오염', '훼손', '훼손', '오염', '터짐', '훼손', '훼손',
       '오염', '훼손', '곰팡이', '훼손', '피스', '훼손', '훼손',
       '훼손', '오염', '훼손', '훼손', '석고수정', '오염',
       '오타공', '오염', '터짐', 

## Submission

In [ ]:
submit = pd.read_csv('drive/MyDrive/Doedal/sample_submission.csv')

In [ ]:
submit['label'] = result

In [ ]:
submit

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,훼손
3,TEST_003,훼손
4,TEST_004,오염
...,...,...
787,TEST_787,훼손
788,TEST_788,터짐
789,TEST_789,오염
790,TEST_790,들뜸


In [ ]:
import chardet

print(chardet.detect(submit['label'][0].encode()))

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [ ]:
# 인코딩 오류 수정하는 코드(자음 모음 분리 합쳐줌)
import unicodedata

for i in range(len(submit['label'])):
  submit['label'][i] = unicodedata.normalize('NFC',submit['label'][i])

In [ ]:
submit['label'][0]

'훼손'

In [ ]:
print(chardet.detect(submit['label'][0].encode()))

{'encoding': 'utf-8', 'confidence': 0.7525, 'language': ''}


In [ ]:
submit.to_csv('drive/MyDrive/Doedal/baseline_submit_3.csv', index=False)